In [1]:
import logging
from datetime import datetime, date, timedelta
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
#from providers.ib.IbTrader import IbTrader
from sqlite import sqliteDB, getTableName
from util import chainList, chart
from strategy import strategy

In [2]:

def backTestRunner(symbols, provider, candle_size, start_date, end_date, init_balance, stratOpts):
  one_day = timedelta(days=1)
  current_date = start_date
  db = sqliteDB()
  db.connect()
  def getTable(symbol):
    return getTableName(provider, symbol, candle_size)

  # Get warm up candles:
  candles = {}
  for symbol in symbols:
    candles[symbol] = db.getLastCandles(getTable(symbol), start_date, stratOpts['numWarmUpCandles'])
  
  logging.debug(f'WARM UP CANDLES:\n{candles["AUDUSD"]}')
  strat = strategy(stratOpts)
  while current_date < end_date:
    print(f'\nCURRENT DATE: {current_date.strftime("%Y-%m-%d")}')
    for symbol in symbols:
        candle = db.getCandle(getTable(symbol), current_date)
        if candle == None:
            print('Not a trading day')
        else:
            candles[symbol] = candles[symbol].append(candle, ignore_index=True)
            logging.debug(f'appended to candles:\n{candles[symbol]}')
            strat.onCandle(candles[symbol])
    #print('\n', datetime.strftime(current_date, '%Y-%m-%d'), symbol, candles[symbol])
    current_date += one_day

  db.disconnect()

In [3]:

logging.basicConfig(level=logging.ERROR)
provider = 'ib'
#symbol = 'EURUSD'
candle_size = '1D'
start_date = datetime(2020, 1, 1)
end_date = datetime(2020, 1, 8)
init_balance = 1000
stratOpts = {
  'tpR': 16,
  'closeOrderAfter': 10,
  'retracementLevelPct': 50,
  'numWarmUpCandles': 1
}
#symbols = ['AUDUSD', 'EURUSD', 'GBPUSD', 'NZDUSD', 'USDCAD', 'USDCHF', 'USDJPY']
symbols = ['AUDUSD']
""" symbols = ['AUDUSD', 'EURUSD', 'GBPUSD', 'NZDUSD', 'USDCAD', 'USDCHF', 'USDJPY',
  'AAPL', 'ADBE', 'AMGN', 'AMZN', 'BABA', 'BKNG', 'CHTR', 'CMCSA', 'COST', 'CSCO',
  'FB', 'FISV', 'GILD', 'GOOGL', 'INTC', 'INTU', 'MA', 'MDLZ', 'MSFT', 'NFLX', 'NVDA',
  'QCOM', 'SBUX', 'TXN', 'WMT',
  'ATVI', 'AMD', 'ALXN', 'ALGN', 'GOOG', 'ADI', 'ANSS', 'AMAT', 'ASML', 'ADSK',
  'ADP', 'BIDU', 'BIIB', 'BMRN', 'AVGO', 'CDNS', 'CDW', 'CERN', 'CHKP', 'CTAS', 'CTXS',
  'CTSH', 'CPRT', 'CSX',
  'DXCM', 'DOCU', 'DLTR', 'EBAY', 'EA', 'EXC', 'EXPE', 'FAST', 'FOX', 'FOXA',
  'IDXX', 'ILMN', 'INCY', 'ISRG', 'JD', 'KDP', 'KLAC', 'LRCX', 'LBTYK', 'LULU',
  'MAR', 'MXIM', 'MELI', 'MCHP', 'MRNA', 'MNST', 'NTES', 'NXPI', 'ORLY', 'PCAR',
  'PAYX', 'PYPL', 'PEP', 'PDD',
  'REGN', 'ROST', 'SGEN', 'SIRI', 'SWKS', 'SPLK', 'SNPS', 'TMUS', 'TTWO', 'TSLA',
  'KHC', 'TCOM', 'ULTA', 'VRSN', 'VRSK', 'VRTX', 'WBA', 'WDAY', 'XEL', 'XLNX', 'ZM'] """

candleDf = backTestRunner(symbols, provider, candle_size, start_date, end_date, init_balance, stratOpts)

CURRENT DATE: 2020-01-01
Not a trading day
CURRENT DATE: 2020-01-02

onCandle:
        date     open      high      low     close  volume
0 2019-12-31  0.69945  0.703205  0.69943  0.701660      -1
1 2020-01-02  0.70189  0.701995  0.69786  0.699305      -1
CURRENT DATE: 2020-01-03

onCandle:
        date     open      high      low     close  volume
0 2019-12-31  0.69945  0.703205  0.69943  0.701660      -1
1 2020-01-02  0.70189  0.701995  0.69786  0.699305      -1
2 2020-01-03  0.69915  0.699275  0.69303  0.694930      -1
CURRENT DATE: 2020-01-04
Not a trading day
CURRENT DATE: 2020-01-05
Not a trading day
CURRENT DATE: 2020-01-06

onCandle:
        date     open      high       low     close  volume
0 2019-12-31  0.69945  0.703205  0.699430  0.701660      -1
1 2020-01-02  0.70189  0.701995  0.697860  0.699305      -1
2 2020-01-03  0.69915  0.699275  0.693030  0.694930      -1
3 2020-01-06  0.69410  0.695775  0.692545  0.693965      -1
CURRENT DATE: 2020-01-07

onCandle:
        date  

In [1]:


def executeStrategy(candleDf, stratOpts):
  tpR = stratOpts['tpR']
  closeOrderAfter = stratOpts['closeOrderAfter']
  retracementLevelPct = stratOpts['retracementLevelPct']
  #print('TP R:', tpR)
  orders = []
  positions = []
  trades = []
  for index, candle in candleDf.iterrows():
    logging.debug(f'CANDLE{candle["date"]}')
    if index > 0:
      prevCandle = candleDf.loc[index-1]
      if prevCandle.engulfing == True:
        logging.debug(f'Previous is ENGULFING. pH:{prevCandle.high} pL:{prevCandle.low} retPct:{retracementLevelPct}')
        retracementLevel = ((prevCandle.high - prevCandle.low) * retracementLevelPct / 100) + prevCandle.low
        sl = prevCandle.low * 0.9995
        #tpAnotherEngCandleLength = candleDf.loc[index-1, 'close'] - candleDf.loc[index-1, 'open'] + candleDf.loc[index-1, 'close']
        static_r_tp = ((retracementLevel - sl) * tpR) + retracementLevel
        if candle.low < retracementLevel:
            logging.debug('Open pos as candle low is below retracement level')
            position = {
              'price_bought': retracementLevel,
              'sl': sl,
              'tp': static_r_tp,
              'date_bought': candle.date
            }
            positions.append(position)
        elif candle.low > retracementLevel:
          logging.debug(f'Place order for when price reaches {retracementLevel}')
          order = {
            'buy_at': retracementLevel,
            'sl': sl,
            'candles_open': 0,
            'placed': candle.date
          }
          order['tp'] = static_r_tp
          orders.append(order)
      if bool(len(orders)): # If there are orders placed, check if we can open position or we have to cancel
        for order in orders:
          if candle.low <= order['buy_at']:
            logging.debug(f'Order placed {order["placed"]} filled. Open pos at {order["buy_at"]}')
            #print('buy')
            position = {
              'price_bought': order['buy_at'],
              'sl': order['sl'],
              'tp': order['tp'],
              'date_bought': candle.date
            }
            order['remove'] = True
            positions.append(position)
          else:
            if order['candles_open'] > closeOrderAfter:
              logging.debug(f'Cancelling order placed on {order["placed"]}')
              order['remove'] = True
            else:
              order['candles_open'] += 1
        for order in orders:
          if 'remove' in order:
            orders.remove(order)
      if bool(len(positions)):
        for position in positions:
          if candle.low <= position['sl']:
            #print('sell sl')
            trade = {
              'price_bought': position['price_bought'],
              'price_sold': position['sl'],
              'date_bought': position['date_bought'],
              'date_sold': candle.date
            }
            trade['profit'] = trade['price_sold'] - trade['price_bought']
            trade['profit_mlt'] = trade['profit']  / trade['price_bought'] + 1
            trade['r'] = (position['tp'] - trade['price_bought']) / (trade['price_bought'] - position['sl'])
            trades.append(trade)
            logging.debug(f'Closing pos opened {position["date_bought"]}. Low is below sl. Loss:{-trade["profit"]}')
            position['remove'] = True
          elif candle.high >= position['tp']:
            trade = {
              'price_bought': position['price_bought'],
              'price_sold': position['tp'],
              'date_bought': position['date_bought'],
              'date_sold': candle.date
            }
            trade['profit'] = trade['price_sold'] - trade['price_bought']
            trade['profit_mlt'] = trade['profit']  / trade['price_bought']  + 1
            trade['r'] = (position['tp'] - trade['price_bought']) / (trade['price_bought'] - position['sl'])
            #print('sell tp')
            trades.append(trade)
            logging.debug(f'Closing pos opened {position["date_bought"]}. High is above tp. Profit:{trade["profit"]} ')
            position['remove'] = True
        for position in positions:
          if 'remove' in position:
            positions.remove(position)
  if bool(len(positions)): # Close any positions that have been left open
    #print('Closing positions still open at the end:')
    for position in positions:
      logging.debug(f'Closing position opened {position["date_bought"]}')
      trade = {
        'price_bought': position['price_bought'],
        'price_sold': candleDf.iloc[-1]['close'],
        'date_bought': position['date_bought'],
        'date_sold': candleDf.iloc[-1]['date']
      }
      trade['profit'] = trade['price_sold'] - trade['price_bought']
      trade['profit_mlt'] = trade['profit']  / trade['price_bought'] + 1
      trade['r'] = (position['tp'] - trade['price_bought']) / (trade['price_bought'] - position['sl'])
      #print('sell tp')
      trades.append(trade)
  return trades

def getBalance(init_balance, trades):
  balance = init_balance
  for trade in trades:
    num_shares = balance / trade['price_bought']
    balance = num_shares * trade['price_sold']
    trade['balance'] = balance
  return balance

def getPerformanceReport(trades):
  def calcs(perf, trade):
    perf['profit'] = perf['profit'] + trade['profit']
    perf['profit_mlt'] = perf['profit_mlt'] * trade['profit_mlt']
    perf['num_wins'] += 1 if trade['profit'] > 0 else 0
    return perf
  if len(trades):
    performance = chainList(trades).reduce(calcs, {'profit': 0, 'profit_mlt': 1, 'num_wins': 0})
    performance['win_rate_pct'] = performance['num_wins'] / len(trades) * 100
    performance['num_trades'] = len(trades)
  else:
    performance = None
  return performance

def overallPerformaceReport(results):
  performance = {}
  for symbol_data in results.values():
    if 'profit' not in performance:
      performance['profit'] = symbol_data['performance']['profit']
      performance['profit_mlt'] = symbol_data['performance']['profit_mlt']
      performance['num_wins'] = symbol_data['performance']['num_wins']
      performance['num_trades'] = symbol_data['performance']['num_trades']
    else:
      performance['profit'] += symbol_data['performance']['profit']
      performance['profit_mlt'] *= symbol_data['performance']['profit_mlt']
      performance['num_wins'] += symbol_data['performance']['num_wins']
      performance['num_trades'] += symbol_data['performance']['num_trades']
  if 'num_trades' in performance: 
    performance['win_rate_pct'] = performance['num_wins'] / performance['num_trades'] * 100
  return pd.DataFrame([performance])




SyntaxError: invalid syntax (<ipython-input-1-ede640874e5f>, line 173)

In [7]:

import plotly.graph_objects as go

def charty(candeDf, scatter={'x': [], 'y': [], 'name': ''}, width=800, height=600, margin=dict(l=50, r=50, b=100, t=100, pad=4)):
    df = candeDf.copy()
    #df['date'] = df['date'].apply(lambda x: datetime.strptime(x, '%Y%m%d'))
    fig = go.Figure(
        data=[
          go.Candlestick(x=df['date'],
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close'],
            name='Candlesticks'),
          go.Scatter(name=scatter['name'], x=scatter['x'], y=scatter['y'], mode='markers', marker={'symbol':'cross', 'size': 6, 'color': 'blue'})
        ])
    config = dict({
        'scrollZoom': True,
        'displaylogo': False,
        'modeBarButtonsToAdd': ['drawline', 'drawopenpath', 'drawclosedpath', 'drawcircle', 'drawrect', 'eraseshape']
    })
    fig.update_layout(
        xaxis_rangeslider_visible=False,
        autosize=False,
        width=width,
        height=height,
        margin=margin,
        dragmode='pan'
      )
    fig.show(config=config)
scatterDots = {'x':candleDf.loc[candleDf['engulfing']]['date'], 'y':(candleDf.loc[candleDf['engulfing']]['open']+candleDf.loc[candleDf['engulfing']]['close'])/2, 'name': 'Engulfing Candles'}

provider = 'ib'
candle_size = '1D'
start_date = datetime(2010, 1, 1)
end_date = datetime(2020, 11, 30)
candleDf = getData(provider, 'AUDUSD', candle_size, start_date, end_date)
candleDf['engulfing'] = getBullishEngulfingCol(candleDf)
charty(candleDf, scatterDots, width=1800, height=800, margin=dict(l=50, r=50, b=50, t=50, pad=4))

In [33]:
candleDf.loc[candleDf['engulfing']]

,date,open,high,low,close,volume,engulfing
36,2020-02-21,1.078325,1.086365,1.078325,1.084765,-1,True
70,2020-04-09,1.085085,1.095180,1.084095,1.092910,-1,True
73,2020-04-14,1.090975,1.098720,1.090445,1.098095,-1,True
84,2020-04-29,1.081880,1.088575,1.081880,1.087450,-1,True
93,2020-05-12,1.080700,1.088545,1.078440,1.084790,-1,True
113,2020-06-09,1.129225,1.136395,1.124105,1.134135,-1,True
117,2020-06-15,1.124300,1.133255,1.122660,1.132325,-1,True
129,2020-07-01,1.123355,1.127525,1.118480,1.125135,-1,True
134,2020-07-08,1.127150,1.135195,1.126240,1.133005,-1,True
141,2020-07-17,1.137850,1.144375,1.137760,1.142770,-1,True
